In [1]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv('heart.csv')

df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [2]:
categorical_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
numerical_cols = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

X = df.drop(columns=['HeartDisease'])
y = df['HeartDisease']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

X = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
def neural_network(input_shape, l2_reg=0):
  model = keras.Sequential([
    keras.Input(shape=(input_shape,)),
    keras.layers.Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=keras.regularizers.l2(l2_reg)),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(l2_reg)),
    keras.layers.Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [7]:
model = neural_network(X_train.shape[1])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

_, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy without L2 regularization: {accuracy:.2f}')

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7391 - loss: 0.6068 - val_accuracy: 0.8478 - val_loss: 0.4310
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - accuracy: 0.8481 - loss: 0.3833 - val_accuracy: 0.8478 - val_loss: 0.3792
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8868 - loss: 0.2993 - val_accuracy: 0.8587 - val_loss: 0.3650
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.8784 - loss: 0.3028 - val_accuracy: 0.8587 - val_loss: 0.3478
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - accuracy: 0.8853 - loss: 0.2875 - val_accuracy: 0.8696 - val_loss: 0.3369
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - accuracy: 0.9042 - loss: 0.2596 - val_accuracy: 0.8533 - val_loss: 0.3443
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.8650 - loss: 0.3047 - val_accuracy: 0.8696 - val_loss: 0.3131
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.8793 - loss: 0.2818 - val_accuracy: 0.85

In [6]:
model_l2 = neural_network(X_train.shape[1], l2_reg=0.05)
history_l2 = model_l2.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

_, accuracy_l2 = model_l2.evaluate(X_test, y_test)
print(f'Accuracy with L2 regularization: {accuracy_l2:.2f}')

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6615 - loss: 3.8372 - val_accuracy: 0.8098 - val_loss: 2.7297
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.8629 - loss: 2.4207 - val_accuracy: 0.8370 - val_loss: 1.7804
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8465 - loss: 1.5953 - val_accuracy: 0.8370 - val_loss: 1.2228
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.8504 - loss: 1.0926 - val_accuracy: 0.8478 - val_loss: 0.8903
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.8560 - loss: 0.8091 - val_accuracy: 0.8315 - val_loss: 0.7221
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.8661 - loss: 0.6326 - val_accuracy: 0.8478 - val_loss: 0.6065
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.8619 - loss: 0.5423 - val_accuracy: 0.8533 - val_loss: 0.5494
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - accuracy: 0.8784 - loss: 0.4745 - val_accuracy: 0.85